<table style="width:100%">
<tr>
<td style="vertical-align:middle; text-align:left;">
<font size="2">
<a href="http://mng.bz/orYv">《从零开始构建大型语言模型》</a>一书的补充代码，作者：<a href="https://sebastianraschka.com">Sebastian Raschka</a><br>
<br>代码仓库：<a href="https://github.com/rasbt/LLMs-from-scratch">https://github.com/rasbt/LLMs-from-scratch</a>
</font>
</td>
<td style="vertical-align:middle; text-align:left;">
<a href="http://mng.bz/orYv"><img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/cover-small.webp" width="100px"></a>
</td>
</tr>
</table>


# 第2章：处理文本数据

本笔记本中使用的包：

In [1]:
from importlib.metadata import version

print("torch version:", version("torch"))
print("tiktoken version:", version("tiktoken"))

torch version: 2.7.1
tiktoken version: 0.9.0


- 本章涵盖数据准备和采样，以使输入数据为LLM做好"准备"

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/01.webp?timestamp=1" width="500px">

## 2.1 理解词嵌入

- 本节没有代码

- 嵌入有很多形式；我们在本书中专注于文本嵌入

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/02.webp" width="500px">

- LLMs在高维空间中使用嵌入（即数千个维度）
- 由于我们无法可视化这样的高维空间（我们人类以1、2或3维思考），下图说明了一个2维嵌入空间

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/03.webp" width="300px">

## 2.2 文本分词

- 在本节中，我们对文本进行分词，这意味着将文本分解为更小的单元，如单个单词和标点符号

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/04.webp" width="300px">

- 加载我们想要处理的原始文本
- [《判决》作者：伊迪丝·华顿](https://en.wikisource.org/wiki/The_Verdict) 是一个公有领域的短篇小说

In [2]:
import os
import urllib.request

if not os.path.exists("the-verdict.txt"):
    url = ("https://raw.githubusercontent.com/rasbt/"
           "LLMs-from-scratch/main/ch02/01_main-chapter-code/"
           "the-verdict.txt")
    file_path = "the-verdict.txt"
    urllib.request.urlretrieve(url, file_path)

- （如果在执行前面的代码单元格时遇到`ssl.SSLCertVerificationError`，可能是由于使用了过时的Python版本；您可以在[GitHub上找到更多信息](https://github.com/rasbt/LLMs-from-scratch/pull/403)）

In [3]:
with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()
    
print("Total number of character:", len(raw_text))
print(raw_text[:99])

Total number of character: 20479
I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no 


- 目标是为LLM对这个文本进行分词和嵌入
- 让我们基于一些简单的示例文本开发一个简单的分词器，然后可以将其应用到上面的文本
- 以下正则表达式将在空白字符上分割

In [4]:
import re

text = "Hello, world. This, is a test."
result = re.split(r'(\s)', text)

print(result)

['Hello,', ' ', 'world.', ' ', 'This,', ' ', 'is', ' ', 'a', ' ', 'test.']


- 我们不仅要在空白字符上分割，还要在逗号和句号上分割，所以让我们修改正则表达式来做到这一点

In [5]:
result = re.split(r'([,.]|\s)', text)

print(result)

['Hello', ',', '', ' ', 'world', '.', '', ' ', 'This', ',', '', ' ', 'is', ' ', 'a', ' ', 'test', '.', '']


- 如我们所见，这会创建空字符串，让我们删除它们

In [6]:
# 从每个项目中去除空白字符，然后过滤掉任何空字符串。
result = [item for item in result if item.strip()]
print(result)

['Hello', ',', 'world', '.', 'This', ',', 'is', 'a', 'test', '.']


- 这看起来很不错，但让我们也处理其他类型的标点符号，如句号、问号等

In [7]:
text = "Hello, world. Is this-- a test?"

result = re.split(r'([,.:;?_!"()\']|--|\s)', text)
result = [item.strip() for item in result if item.strip()]
print(result)

['Hello', ',', 'world', '.', 'Is', 'this', '--', 'a', 'test', '?']


- 这很不错，我们现在准备将这种分词应用到原始文本

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/05.webp" width="350px">

In [8]:
preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', raw_text)
preprocessed = [item.strip() for item in preprocessed if item.strip()]
print(preprocessed[:30])

['I', 'HAD', 'always', 'thought', 'Jack', 'Gisburn', 'rather', 'a', 'cheap', 'genius', '--', 'though', 'a', 'good', 'fellow', 'enough', '--', 'so', 'it', 'was', 'no', 'great', 'surprise', 'to', 'me', 'to', 'hear', 'that', ',', 'in']


- 让我们计算令牌的总数

In [9]:
print(len(preprocessed))

4690


## 2.3 将令牌转换为令牌ID

- 下一步是将文本令牌转换为令牌ID，这些ID可以通过嵌入层处理

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/06.webp" width="500px">

- 从这些令牌中，我们现在可以构建一个由所有唯一令牌组成的词汇表

In [10]:
all_words = sorted(set(preprocessed))
vocab_size = len(all_words)

print(vocab_size)

1130


In [11]:
vocab = {token:integer for integer,token in enumerate(all_words)}

- 下面是此词汇表中的前50个条目：

In [12]:
for i, item in enumerate(vocab.items()):
    print(item)
    if i >= 50:
        break

('!', 0)
('"', 1)
("'", 2)
('(', 3)
(')', 4)
(',', 5)
('--', 6)
('.', 7)
(':', 8)
(';', 9)
('?', 10)
('A', 11)
('Ah', 12)
('Among', 13)
('And', 14)
('Are', 15)
('Arrt', 16)
('As', 17)
('At', 18)
('Be', 19)
('Begin', 20)
('Burlington', 21)
('But', 22)
('By', 23)
('Carlo', 24)
('Chicago', 25)
('Claude', 26)
('Come', 27)
('Croft', 28)
('Destroyed', 29)
('Devonshire', 30)
('Don', 31)
('Dubarry', 32)
('Emperors', 33)
('Florence', 34)
('For', 35)
('Gallery', 36)
('Gideon', 37)
('Gisburn', 38)
('Gisburns', 39)
('Grafton', 40)
('Greek', 41)
('Grindle', 42)
('Grindles', 43)
('HAD', 44)
('Had', 45)
('Hang', 46)
('Has', 47)
('He', 48)
('Her', 49)
('Hermia', 50)


- 下面，我们使用一个小词汇表来说明短示例文本的分词：

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/07.webp?123" width="500px">

- 现在将所有内容整合到一个分词器类中

In [13]:
class SimpleTokenizerV1:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = {i:s for s,i in vocab.items()}
    
    def encode(self, text):
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)
                                
        preprocessed = [
            item.strip() for item in preprocessed if item.strip()
        ]
        ids = [self.str_to_int[s] for s in preprocessed]
        return ids
        
    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        # Replace spaces before the specified punctuations
        text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)
        return text

- `encode`函数将文本转换为令牌ID
- `decode`函数将令牌ID转换回文本

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/08.webp?123" width="500px">

- 我们可以使用分词器将文本编码（即分词）为整数
- 这些整数然后可以（稍后）作为LLM的输入进行嵌入

In [14]:
tokenizer = SimpleTokenizerV1(vocab)

text = """"It's the last he painted, you know," 
           Mrs. Gisburn said with pardonable pride."""
ids = tokenizer.encode(text)
print(ids)

[1, 56, 2, 850, 988, 602, 533, 746, 5, 1126, 596, 5, 1, 67, 7, 38, 851, 1108, 754, 793, 7]


- 我们可以将整数解码回文本

In [15]:
tokenizer.decode(ids)

'" It\' s the last he painted, you know," Mrs. Gisburn said with pardonable pride.'

In [16]:
tokenizer.decode(tokenizer.encode(text))

'" It\' s the last he painted, you know," Mrs. Gisburn said with pardonable pride.'

## 2.4 添加特殊上下文令牌

- 为未知单词添加一些"特殊"令牌并表示文本结束是很有用的

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/09.webp?123" width="500px">

- 一些分词器使用特殊令牌来帮助LLM获得额外的上下文
- 其中一些特殊令牌是
  - `[BOS]`（序列开始）标记文本的开始
  - `[EOS]`（序列结束）标记文本结束的位置（这通常用于连接多个不相关的文本，例如两篇不同的维基百科文章或两本不同的书等）
  - `[PAD]`（填充）如果我们使用大于1的批次大小训练LLM（我们可能包含多个不同长度的文本；使用填充令牌，我们将较短的文本填充到最长长度，使所有文本具有相等的长度）
- `[UNK]`表示不包含在词汇表中的单词

- 请注意，GPT-2不需要上述提到的任何这些令牌，只使用`<|endoftext|>`令牌来降低复杂性
- `<|endoftext|>`类似于上面提到的`[EOS]`令牌
- GPT也使用`<|endoftext|>`进行填充（因为我们在批量输入上训练时通常使用掩码，我们不会关注填充的令牌，所以这些令牌是什么并不重要）
- GPT-2不使用`<UNK>`令牌处理词汇表外的单词；相反，GPT-2使用字节对编码（BPE）分词器，它将单词分解为子词单元，我们将在后面的章节中讨论



- 我们在两个独立的文本源之间使用`<|endoftext|>`令牌：

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/10.webp" width="500px">

- 让我们看看如果我们对以下文本进行分词会发生什么：

In [17]:
tokenizer = SimpleTokenizerV1(vocab)

text = "Hello, do you like tea. Is this-- a test?"

tokenizer.encode(text)

KeyError: 'Hello'

- 上面的代码会产生错误，因为单词"Hello"不包含在词汇表中
- 为了处理这种情况，我们可以向词汇表添加特殊令牌，如`"<|unk|>"`来表示未知单词
- 既然我们已经在扩展词汇表，让我们添加另一个名为`"<|endoftext|>"`的令牌，它在GPT-2训练中用于表示文本的结束（它也用于连接的文本之间，比如我们的训练数据集由多篇文章、书籍等组成）

In [18]:
all_tokens = sorted(list(set(preprocessed)))
all_tokens.extend(["<|endoftext|>", "<|unk|>"])

vocab = {token:integer for integer,token in enumerate(all_tokens)}

In [19]:
len(vocab.items())

1132

In [20]:
for i, item in enumerate(list(vocab.items())[-5:]):
    print(item)

('younger', 1127)
('your', 1128)
('yourself', 1129)
('<|endoftext|>', 1130)
('<|unk|>', 1131)


- 我们还需要相应地调整分词器，使其知道何时以及如何使用新的`<unk>`令牌

In [22]:
class SimpleTokenizerV2:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = { i:s for s,i in vocab.items()}
    
    def encode(self, text):
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)
        preprocessed = [item.strip() for item in preprocessed if item.strip()]
        preprocessed = [
            item if item in self.str_to_int 
            else "<|unk|>" for item in preprocessed
        ]

        ids = [self.str_to_int[s] for s in preprocessed]
        return ids
        
    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        # Replace spaces before the specified punctuations
        text = re.sub(r'\s+([,.:;?!"()\'])', r'\1', text)
        return text

让我们尝试使用修改后的分词器对文本进行分词：

In [23]:
tokenizer = SimpleTokenizerV2(vocab)

text1 = "Hello, do you like tea?"
text2 = "In the sunlit terraces of the palace."

text = " <|endoftext|> ".join((text1, text2))

print(text)

Hello, do you like tea? <|endoftext|> In the sunlit terraces of the palace.


In [24]:
tokenizer.encode(text)

[1131, 5, 355, 1126, 628, 975, 10, 1130, 55, 988, 956, 984, 722, 988, 1131, 7]

In [25]:
tokenizer.decode(tokenizer.encode(text))

'<|unk|>, do you like tea? <|endoftext|> In the sunlit terraces of the <|unk|>.'

## 2.5 字节对编码

- GPT-2使用字节对编码（BPE）作为其分词器
- 它允许模型将不在其预定义词汇表中的单词分解为更小的子词单元甚至单个字符，使其能够处理词汇表外的单词
- 例如，如果GPT-2的词汇表没有单词"unfamiliarword"，它可能会将其分词为["unfam", "iliar", "word"]或其他子词分解，这取决于其训练的BPE合并
- 原始的BPE分词器可以在这里找到：[https://github.com/openai/gpt-2/blob/master/src/encoder.py](https://github.com/openai/gpt-2/blob/master/src/encoder.py)
- 在本章中，我们使用OpenAI开源[tiktoken](https://github.com/openai/tiktoken)库中的BPE分词器，它在Rust中实现其核心算法以提高计算性能
- 我在[./bytepair_encoder](../02_bonus_bytepair-encoder)中创建了一个笔记本，并排比较了这两种实现（tiktoken在示例文本上快了约5倍）

In [31]:
# pip install tiktoken

In [26]:
import importlib
import tiktoken

print("tiktoken version:", importlib.metadata.version("tiktoken"))

tiktoken version: 0.9.0


In [27]:
tokenizer = tiktoken.get_encoding("gpt2")

In [28]:
text = (
    "Hello, do you like tea? <|endoftext|> In the sunlit terraces"
     "of someunknownPlace."
)

integers = tokenizer.encode(text, allowed_special={"<|endoftext|>"})

print(integers)

[15496, 11, 466, 345, 588, 8887, 30, 220, 50256, 554, 262, 4252, 18250, 8812, 2114, 1659, 617, 34680, 27271, 13]


In [29]:
strings = tokenizer.decode(integers)

print(strings)

Hello, do you like tea? <|endoftext|> In the sunlit terracesof someunknownPlace.


- BPE分词器将未知单词分解为子词和单个字符：

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/11.webp" width="300px">

## 2.6 滑动窗口数据采样

- 我们训练LLM一次生成一个单词，所以我们想要相应地准备训练数据，其中序列中的下一个单词代表要预测的目标：

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/12.webp" width="400px">

In [30]:
with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

enc_text = tokenizer.encode(raw_text)
print(len(enc_text))

5145


- 对于每个文本块，我们需要输入和目标
- 由于我们希望模型预测下一个单词，目标是输入向右移动一个位置

In [31]:
enc_sample = enc_text[50:]

In [54]:
context_size = 4

x = enc_sample[:context_size]
y = enc_sample[1:context_size+1]

print(f"x: {x}")
print(f"y:      {y}")

x: [290, 4920, 2241, 287]
y:      [4920, 2241, 287, 257]


- 逐个预测看起来如下：

In [33]:
for i in range(1, context_size+1):
    context = enc_sample[:i]
    desired = enc_sample[i]

    print(context, "---->", desired)

[290] ----> 4920
[290, 4920] ----> 2241
[290, 4920, 2241] ----> 287
[290, 4920, 2241, 287] ----> 257


In [34]:
for i in range(1, context_size+1):
    context = enc_sample[:i]
    desired = enc_sample[i]

    print(tokenizer.decode(context), "---->", tokenizer.decode([desired]))

 and ---->  established
 and established ---->  himself
 and established himself ---->  in
 and established himself in ---->  a


- 我们将在涵盖注意力机制后的后续章节中处理下一个单词预测
- 现在，我们实现一个简单的数据加载器，它遍历输入数据集并返回输入和目标，目标向右移动一位

- 安装并导入PyTorch（安装提示请参见附录A）

In [35]:
import torch
print("PyTorch version:", torch.__version__)

PyTorch version: 2.7.1+cpu


- 我们使用滑动窗口方法，将位置改变+1：

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/13.webp?123" width="500px">

- 创建数据集和数据加载器，从输入文本数据集中提取块

In [36]:
from torch.utils.data import Dataset, DataLoader


class GPTDatasetV1(Dataset):
    def __init__(self, txt, tokenizer, max_length, stride):
        self.input_ids = []
        self.target_ids = []

        # Tokenize the entire text
        token_ids = tokenizer.encode(txt, allowed_special={"<|endoftext|>"})
        assert len(token_ids) > max_length, "Number of tokenized inputs must at least be equal to max_length+1"

        # Use a sliding window to chunk the book into overlapping sequences of max_length
        for i in range(0, len(token_ids) - max_length, stride):
            input_chunk = token_ids[i:i + max_length]
            target_chunk = token_ids[i + 1: i + max_length + 1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.target_ids[idx]

In [37]:
def create_dataloader_v1(txt, batch_size=4, max_length=256, 
                         stride=128, shuffle=True, drop_last=True,
                         num_workers=0):

    # Initialize the tokenizer
    tokenizer = tiktoken.get_encoding("gpt2")

    # Create dataset
    dataset = GPTDatasetV1(txt, tokenizer, max_length, stride)

    # Create dataloader
    dataloader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        drop_last=drop_last,
        num_workers=num_workers
    )

    return dataloader

- 让我们用批次大小为1的数据加载器测试上下文大小为4的LLM：

In [38]:
with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

In [39]:
dataloader = create_dataloader_v1(
    raw_text, batch_size=1, max_length=4, stride=1, shuffle=False
)

data_iter = iter(dataloader)
first_batch = next(data_iter)
print(first_batch)

[tensor([[  40,  367, 2885, 1464]]), tensor([[ 367, 2885, 1464, 1807]])]


In [40]:
second_batch = next(data_iter)
print(second_batch)

[tensor([[ 367, 2885, 1464, 1807]]), tensor([[2885, 1464, 1807, 3619]])]


- 使用步长等于上下文长度（这里：4）的示例如下所示：

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/14.webp" width="500px">

- 我们也可以创建批量输出
- 注意我们在这里增加了步长，这样批次之间就没有重叠，因为更多的重叠可能导致过拟合增加

In [41]:
dataloader = create_dataloader_v1(raw_text, batch_size=8, max_length=4, stride=4, shuffle=False)

data_iter = iter(dataloader)
inputs, targets = next(data_iter)
print("Inputs:\n", inputs)
print("\nTargets:\n", targets)

Inputs:
 tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])

Targets:
 tensor([[  367,  2885,  1464,  1807],
        [ 3619,   402,   271, 10899],
        [ 2138,   257,  7026, 15632],
        [  438,  2016,   257,   922],
        [ 5891,  1576,   438,   568],
        [  340,   373,   645,  1049],
        [ 5975,   284,   502,   284],
        [ 3285,   326,    11,   287]])


## 2.7 创建令牌嵌入

- 数据已经几乎为LLM准备好了
- 但最后让我们使用嵌入层将令牌嵌入到连续向量表示中
- 通常，这些嵌入层是LLM本身的一部分，并在模型训练期间更新（训练）

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/15.webp" width="400px">

- 假设我们有以下四个输入示例，输入ID为2、3、5和1（分词后）：

In [42]:
input_ids = torch.tensor([2, 3, 5, 1])

- 为了简单起见，假设我们有一个只有6个单词的小词汇表，我们想要创建大小为3的嵌入：

In [43]:
vocab_size = 6
output_dim = 3

torch.manual_seed(123)
embedding_layer = torch.nn.Embedding(vocab_size, output_dim)

- 这将产生一个6x3的权重矩阵：

In [44]:
print(embedding_layer.weight)

Parameter containing:
tensor([[ 0.3374, -0.1778, -0.1690],
        [ 0.9178,  1.5810,  1.3010],
        [ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-1.1589,  0.3255, -0.6315],
        [-2.8400, -0.7849, -1.4096]], requires_grad=True)


- 对于那些熟悉独热编码的人来说，上面的嵌入层方法本质上只是实现独热编码后跟全连接层中矩阵乘法的更有效方法，这在[./embedding_vs_matmul](../03_bonus_embedding-vs-matmul)的补充代码中有描述
- 因为嵌入层只是一个更有效的实现，等价于独热编码和矩阵乘法方法，它可以被视为一个可以通过反向传播优化的神经网络层

- 要将ID为3的令牌转换为3维向量，我们执行以下操作：

In [45]:
print(embedding_layer(torch.tensor([3])))

tensor([[-0.4015,  0.9666, -1.1481]], grad_fn=<EmbeddingBackward0>)


- 注意上面是`embedding_layer`权重矩阵中的第4行
- 要嵌入上面所有四个`input_ids`值，我们执行

In [46]:
print(embedding_layer(input_ids))

tensor([[ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-2.8400, -0.7849, -1.4096],
        [ 0.9178,  1.5810,  1.3010]], grad_fn=<EmbeddingBackward0>)


- 嵌入层本质上是一个查找操作：

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/16.webp?123" width="500px">

- **您可能对比较嵌入层与常规线性层的奖励内容感兴趣：[../03_bonus_embedding-vs-matmul](../03_bonus_embedding-vs-matmul)**

## 2.8 编码词位置

- 嵌入层将ID转换为相同的向量表示，无论它们在输入序列中的位置如何：

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/17.webp" width="400px">

- 位置嵌入与令牌嵌入向量结合，形成大型语言模型的输入嵌入：

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/18.webp" width="500px">

- 字节对编码器的词汇表大小为50,257：
- 假设我们想要将输入令牌编码为256维向量表示：

In [47]:
vocab_size = 50257
output_dim = 256

token_embedding_layer = torch.nn.Embedding(vocab_size, output_dim)

- 如果我们从数据加载器中采样数据，我们将每个批次中的令牌嵌入到256维向量中
- 如果我们有批次大小为8，每个有4个令牌，这将产生一个8 x 4 x 256的张量：

In [48]:
max_length = 4
dataloader = create_dataloader_v1(
    raw_text, batch_size=8, max_length=max_length,
    stride=max_length, shuffle=False
)
data_iter = iter(dataloader)
inputs, targets = next(data_iter)

In [55]:
print("Token IDs:\n", inputs)
print("\nInputs shape:\n", inputs.shape)

Token IDs:
 tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])

Inputs shape:
 torch.Size([8, 4])


In [49]:
token_embeddings = token_embedding_layer(inputs)
print(token_embeddings.shape)

# 取消注释并执行以下行以查看嵌入的样子
# print(token_embeddings)

torch.Size([8, 4, 256])


- GPT-2使用绝对位置嵌入，所以我们只需创建另一个嵌入层：

In [50]:
context_length = max_length
pos_embedding_layer = torch.nn.Embedding(context_length, output_dim)

# 取消注释并执行以下行以查看嵌入层权重的样子
# print(pos_embedding_layer.weight)

In [51]:
pos_embeddings = pos_embedding_layer(torch.arange(max_length))
print(pos_embeddings.shape)

# 取消注释并执行以下行以查看嵌入的样子
# print(pos_embeddings)

torch.Size([4, 256])


- 要创建LLM中使用的输入嵌入，我们只需将令牌和位置嵌入相加：

In [52]:
input_embeddings = token_embeddings + pos_embeddings
print(input_embeddings.shape)

# 取消注释并执行以下行以查看嵌入的样子
# print(input_embeddings)

torch.Size([8, 4, 256])


- 在输入处理工作流程的初始阶段，输入文本被分割成单独的令牌
- 在此分割之后，这些令牌基于预定义的词汇表转换为令牌ID：

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/19.webp" width="400px">

# 总结和要点

请参阅[./dataloader.ipynb](./dataloader.ipynb)代码笔记本，这是我们在本章中实现的数据加载器的简洁版本，在即将到来的章节中训练GPT模型时将需要用到。

请参阅[./exercise-solutions.ipynb](./exercise-solutions.ipynb)获取练习解答。

如果您有兴趣了解如何从零开始实现和训练GPT-2分词器，请参阅[从零开始的字节对编码（BPE）分词器](../02_bonus_bytepair-encoder/compare-bpe-tiktoken.ipynb)笔记本。